**Training toxicity classifier model**

In [ ]:
! pip install -q kaggle

Upload kaggle.json

In [ ]:
from google.colab import files
files.upload()

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

Download dataset

In [ ]:
! kaggle competitions download -c 'jigsaw-toxic-comment-classification-challenge'
! unzip 'jigsaw-toxic-comment-classification-challenge.zip'

In [ ]:
! unzip 'train.csv.zip'
! unzip 'test.csv.zip'
! unzip 'test_labels.csv.zip'

Download pretrained glove word embedding

In [ ]:
!wget https://nlp.stanford.edu/data/glove.twitter.27B.zip
!unzip -q glove.twitter.27B.zip

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization, Embedding
from tensorflow.keras import layers

Load and preprocess dataset

In [ ]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_test_labels = pd.read_csv('test_labels.csv')

## combine df_test and its labels then throw away rows with -1 values
df_test_labels_normalized = df_test_labels[df_test_labels['toxic']!=-1]
df_test_normalized = df_test.set_index('id').join(df_test_labels_normalized.set_index('id'), how='right')

In [ ]:
feature = ['comment_text']
target = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

## convert into tf.data.Dataset
train_data = tf.data.Dataset.from_tensor_slices((df_train[feature], df_train[target]))
test_data = tf.data.Dataset.from_tensor_slices((
    df_test_normalized[feature],
    df_test_normalized[target]
))

Use TextVectorization to convert text to sequences

In [ ]:
vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200, ## max_tokens denotes number of words to be tokenized
                               pad_to_max_tokens=True)

vectorizer.adapt(train_data.map(lambda x, y: x).batch(2000)) ## use .map() to get the input only since in the dataset there are input and label

voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

Load pretrained word embedding and put it into dictionary

In [ ]:
path_to_glove_file = "glove.twitter.27B.200d.txt"

embeddings_index = {}
with open(path_to_glove_file) as f:
  for line in f:
    word, coefs = line.split(maxsplit=1)
    coefs = np.fromstring(coefs, "f", sep=" ")
    embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Only use words from pretrained word embedding that exist in vectorizer.get_vocabulary()

In [ ]:
num_tokens = len(voc) + 2  ## TextVectorization already includes OOV and padding, but pretrained glove file also includes OOV and padding so we add 2
embedding_dim = 200  ## 200 as dimension comes from pretrained word embedding
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    # Words not found in embedding index will be all-zeros.
    # This includes the representation for "padding" and "OOV"
    embedding_matrix[i] = embedding_vector
    hits += 1
  else:
    misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Initialize embedding layer with 20000 of pretrained embedding vectors

In [ ]:
embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

Create sequential model

In [ ]:
model = tf.keras.Sequential([
    layers.Input(shape=(200,), dtype='int64'),
    embedding_layer,
    layers.Bidirectional(layers.GRU(128, return_sequences=False)),
    layers.Dense(128, activation='relu'),
    layers.Dense(256, activation='relu'),
    layers.Dense(6, activation='sigmoid')
])

In [ ]:
model.summary()

Transform train data and test data from raw text to sequences

In [ ]:
## define function to convert raw text to sequences
def to_sequence(x, y):
  return vectorizer(x), y

## batch, cache, and prefetch
## set batch of 512, since we have big vram and so we can better utilize the GPU
## batch size tradeoff:
## - big batch -> better GPU utilization -> faster training time -> lower accuracy
## - small batch -> worse GPU utilization -> slower training time -> higher accuracy
train_data = train_data.batch(512).map(to_sequence).cache().prefetch(tf.data.AUTOTUNE) 
test_data = test_data.batch(512).map(to_sequence).cache().prefetch(tf.data.AUTOTUNE)  

Compile and train model

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(), 
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.0006),
              metrics=[tf.keras.metrics.BinaryAccuracy()]) 
                                                                  
history = model.fit(train_data,
                    epochs=8, 
                    validation_data=test_data,
                    verbose=1)


Use greedy search to find out the best threshold that yields the best accuracy for each label

In [ ]:
## make prediction to all train and test dataset
train_prediction = model.predict(train_data.map(lambda x, y: x))
test_prediction = model.predict(test_data.map(lambda x, y: x))

train_accuracy_scores = []
test_accuracy_scores = []
thresholds = []

## iterate all 6 labels
for idx, tgt in enumerate(target):
  best_threshold = 0
  max_train_accuracy = 0
  max_test_accuracy = 0

  ## check threshold from 0.01 to 0.99, with 0.01 step
  for threshold in np.arange(0.01, 1, 0.01): 
    train_accuracy = tf.keras.metrics.BinaryAccuracy(threshold=threshold)
    test_accuracy = tf.keras.metrics.BinaryAccuracy(threshold=threshold)

    ## [:, idx] to take slice of 1 label only
    ## use data from pandas dataframe since it's not batched
    train_accuracy.update_state(df_train[tgt], train_prediction[:,idx])
    test_accuracy.update_state(df_test_normalized[tgt], test_prediction[:,idx])

    train_accuracy = train_accuracy.result().numpy()
    test_accuracy = test_accuracy.result().numpy()

    ## get the best accuracy score based on train data
    if train_accuracy > max_train_accuracy:
      max_test_accuracy = test_accuracy
      max_train_accuracy = train_accuracy
      best_threshold = threshold

  print('label: ', tgt)
  print('train_accuracy: ', train_accuracy)
  print('test_accuracy: ', test_accuracy)
  print('threshold: ', best_threshold, '\n')

  ## store f1 score of train and test data, also store best threshold
  train_accuracy_scores.append(train_accuracy)
  test_accuracy_scores.append(test_accuracy)
  thresholds.append(best_threshold)

print('average train accuracy: ', np.mean(train_accuracy_scores))
print('average test accuracy: ', np.mean(test_accuracy_scores))

Try predicting with the model

In [ ]:
np.set_printoptions(precision=8, suppress=True)  ## print in decimal number, not scientific
seq = vectorizer([["nigga"]]) ## turn text into sequence first using vectorizer
prediction = model.predict(seq)
print(target)   ## print all label names
print(prediction > thresholds)    ## print prediction values as true or false according to thresholds
print(prediction)   ## print prediction value

Append TextVectorization layer to the model, so we don't need to do separate preprocessing and can directly input raw text to the model

In [ ]:
# Start by creating an explicit input layer. It needs to have a shape of  
# (1,) (because we need to guarantee that there is exactly one string  
# input per batch), and the dtype needs to be 'string'.
end_to_end_model = tf.keras.Sequential([
    tf.keras.Input(shape=(1,), dtype=tf.string),
    vectorizer,
    model
])

end_to_end_model.summary()

Try the end-to-end model

In [ ]:
end_to_end_model.predict([["i will smack your face"]])  ## can directly input raw text, no need vectorizer

Save the model

In [ ]:
end_to_end_model.save('./saved_model/end-to-end')  ## model with TextVectorization layer

Save to google drive

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')
# ! cp -r saved_model gdrive/MyDrive

Load from google drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
! cp -r gdrive/MyDrive/saved_model saved_model 

Try to load the saved models

In [ ]:
loaded_end_to_end_model = tf.keras.models.load_model('saved_model/end-to-end')

Make prediction with loaded models

In [ ]:
print('loaded_end_to_end_model: ', loaded_end_to_end_model.predict( [["i will smack your face"]] ))